## Prepare

In [1]:
def recalibrate(p,w):
    q = p/(p+(1-p)/w)
    return q

## Downsampling

In [2]:
import numpy as np
import pandas as pd
Train_df = pd.read_csv('/project/Data/train_newfeature.csv')
Valid_df = pd.read_csv('/project/Data/valid_newfeature.csv')

In [4]:
#Samp_per should be in 0.001,0.01,0.025,0.05,0.075,0.1
samps = [0.001,0.01,0.025,0.05,0.075,0.1]
# samps = [0.001,0.01,0.025,0.05,0.075,0.1]
files = []

In [5]:
for Samp_per in samps:
    #sample 10 times
    for i in range(10):    
        # bids which click =1
        df1 = Train_df[Train_df['click']==1]
        # bids which click =0
        df0 = Train_df[Train_df['click']==0]
        # get click =0 data, down sampling
        df2=df0.sample(frac=Samp_per)
        # combine % of click =0 data and all click=1, down sampling
        combined_df = pd.concat([df1, df2],axis=0)
        combined_df.head()
        combined_df.shape
        #new trainning set shape is(62523, 127)
        combined_name = '/project/tffm-master/csvs/'+'train_sample'+str(Samp_per)+'_'+str(i)+'.csv'
        files.append([combined_name,Samp_per,i])
        combined_df.to_csv(combined_name, encoding='utf-8', index=False)
        print(combined_name+' is sampled.')
    

/project/tffm-master/csvs/train_sample0.001_0.csv is sampled.
/project/tffm-master/csvs/train_sample0.001_1.csv is sampled.
/project/tffm-master/csvs/train_sample0.001_2.csv is sampled.
/project/tffm-master/csvs/train_sample0.001_3.csv is sampled.
/project/tffm-master/csvs/train_sample0.001_4.csv is sampled.
/project/tffm-master/csvs/train_sample0.001_5.csv is sampled.
/project/tffm-master/csvs/train_sample0.001_6.csv is sampled.
/project/tffm-master/csvs/train_sample0.001_7.csv is sampled.
/project/tffm-master/csvs/train_sample0.001_8.csv is sampled.
/project/tffm-master/csvs/train_sample0.001_9.csv is sampled.
/project/tffm-master/csvs/train_sample0.01_0.csv is sampled.
/project/tffm-master/csvs/train_sample0.01_1.csv is sampled.
/project/tffm-master/csvs/train_sample0.01_2.csv is sampled.
/project/tffm-master/csvs/train_sample0.01_3.csv is sampled.
/project/tffm-master/csvs/train_sample0.01_4.csv is sampled.
/project/tffm-master/csvs/train_sample0.01_5.csv is sampled.
/project/tffm-

In [5]:
# files[0][1]

0.001

## model

In [10]:
from tffm import TFFMClassifier
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score


modelfm = TFFMClassifier(
    order=3, 
    rank=10, 
    optimizer=tf.train.AdamOptimizer(learning_rate=0.001), 
    n_epochs=30, 
    batch_size=1024,
    init_std=0.001,
    reg=0.01,
    input_type='sparse',
    seed=42
)

#modelfit(xgb4, train, predictors)

## train

In [11]:
#from sklearn.externals import joblib
cols=list(Train_df.columns)[1:]

In [12]:
Valid_df['slotheight']=Valid_df['slotheight']/10
Valid_df['slotwidth']=Valid_df['slotwidth']/100
Valid_df['slotprice']=Valid_df['slotprice']/100

In [13]:
valid_results = []

In [16]:
import scipy.sparse as sp

#your own location
#loc = '/project/tffm-master/csvs/'

#file[combined_name,Samp_per,i]
for file in files:
    
    #read
    #fileloc = loc+file[0]
    fileloc = file[0]
    combined_df = pd.read_csv(fileloc)
    print(fileloc+' read.')
    
    combined_df['slotheight']=combined_df['slotheight']/10
    combined_df['slotwidth']=combined_df['slotwidth']/100
    combined_df['slotprice']=combined_df['slotprice']/100
    
    train_y = combined_df['click']
    train_x = combined_df[cols]
    
    train_x=train_x.values
    train_y=train_y.values
    
    #train model
    X = sp.csr_matrix(train_x)
    y = train_y
    
    model = modelfm
    model.fit(X, y)
    #save model
    #filename = file[0]+'_xg.sav'
    #joblib.dump(model, filename)
    
    
    
    #validation set
    valid_x = Valid_df[cols]
    valid_x = sp.csr_matrix(valid_x.values)

    
    valid_predictions = model.predict_proba(valid_x)
    valid_predictions_class = model.predict(valid_x)
    #save valid result
    valid_result = pd.DataFrame(columns=['click', 'click_pred_per', 'click_pred_class'])  
    valid_result['click']= Valid_df['click'].copy()
    valid_result['click_pred_per'] = valid_predictions[:,1]
    valid_result['click_pred_per']=recalibrate(valid_result['click_pred_per'],file[1])
    valid_result['click_pred_class'] = valid_predictions_class
    valid_results.append(valid_result)
    print('valid_result on '+file[0]+' saved.')
    

/project/tffm-master/csvs/train_sample0.001_0.csv read.
valid_result on /project/tffm-master/csvs/train_sample0.001_0.csv saved.
/project/tffm-master/csvs/train_sample0.001_1.csv read.
valid_result on /project/tffm-master/csvs/train_sample0.001_1.csv saved.
/project/tffm-master/csvs/train_sample0.001_2.csv read.
valid_result on /project/tffm-master/csvs/train_sample0.001_2.csv saved.
/project/tffm-master/csvs/train_sample0.001_3.csv read.
valid_result on /project/tffm-master/csvs/train_sample0.001_3.csv saved.
/project/tffm-master/csvs/train_sample0.001_4.csv read.
valid_result on /project/tffm-master/csvs/train_sample0.001_4.csv saved.
/project/tffm-master/csvs/train_sample0.001_5.csv read.
valid_result on /project/tffm-master/csvs/train_sample0.001_5.csv saved.
/project/tffm-master/csvs/train_sample0.001_6.csv read.


/project/tffm-master/tffm/utils.py:195: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


valid_result on /project/tffm-master/csvs/train_sample0.001_6.csv saved.
/project/tffm-master/csvs/train_sample0.001_7.csv read.
valid_result on /project/tffm-master/csvs/train_sample0.001_7.csv saved.
/project/tffm-master/csvs/train_sample0.001_8.csv read.
valid_result on /project/tffm-master/csvs/train_sample0.001_8.csv saved.
/project/tffm-master/csvs/train_sample0.001_9.csv read.
valid_result on /project/tffm-master/csvs/train_sample0.001_9.csv saved.
/project/tffm-master/csvs/train_sample0.01_0.csv read.
valid_result on /project/tffm-master/csvs/train_sample0.01_0.csv saved.
/project/tffm-master/csvs/train_sample0.01_1.csv read.
valid_result on /project/tffm-master/csvs/train_sample0.01_1.csv saved.
/project/tffm-master/csvs/train_sample0.01_2.csv read.
valid_result on /project/tffm-master/csvs/train_sample0.01_2.csv saved.
/project/tffm-master/csvs/train_sample0.01_3.csv read.
valid_result on /project/tffm-master/csvs/train_sample0.01_3.csv saved.
/project/tffm-master/csvs/train_

In [20]:
i=0
for file in files:
    valid_result_name = file[0]+'_result.csv'
    valid_results[i].to_csv(valid_result_name, encoding='utf-8', index=False)    
    i=i+1
    print('valid_result on '+file[0]+' saved.')
    

valid_result on /project/tffm-master/csvs/train_sample0.001_0.csv saved.
valid_result on /project/tffm-master/csvs/train_sample0.001_1.csv saved.
valid_result on /project/tffm-master/csvs/train_sample0.001_2.csv saved.
valid_result on /project/tffm-master/csvs/train_sample0.001_3.csv saved.
valid_result on /project/tffm-master/csvs/train_sample0.001_4.csv saved.
valid_result on /project/tffm-master/csvs/train_sample0.001_5.csv saved.
valid_result on /project/tffm-master/csvs/train_sample0.001_6.csv saved.
valid_result on /project/tffm-master/csvs/train_sample0.001_7.csv saved.
valid_result on /project/tffm-master/csvs/train_sample0.001_8.csv saved.
valid_result on /project/tffm-master/csvs/train_sample0.001_9.csv saved.
valid_result on /project/tffm-master/csvs/train_sample0.01_0.csv saved.
valid_result on /project/tffm-master/csvs/train_sample0.01_1.csv saved.
valid_result on /project/tffm-master/csvs/train_sample0.01_2.csv saved.
valid_result on /project/tffm-master/csvs/train_sample

In [72]:
#valid_results[50][valid_results[50].click == 1]

## Find Base Bid

In [23]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import mpld3
import numpy as np

In [24]:
df_valid = pd.read_csv("/project/Data/validation.csv")

In [25]:
# parameters
total_budget = 6250000
payprice = df_valid['payprice'].as_matrix()
Train_df_qr = df_valid.copy()
avgCTR = 1793/2430981 #train set
#tuning...

In [26]:
def generate_bidprice(base_bid, test_predictions):
    avgCTR = 1793/2430981#train set
    bidprices = base_bid*test_predictions/avgCTR
    return bidprices

In [27]:
def Prob3(Train_df,payprice,base_bid,test_predictions):
    Train_Const_summ = pd.DataFrame({'Clicks':'', 'Spend':'', 'CTR':'', 'CPM':'', 'eCPC':''}, index=[0])
    
    #if count_break = 5 break
    count_break = 0
    
    current_budget = total_budget
    Train_df['win'] = 0 #initialize win label
    bidprices = generate_bidprice(base_bid, test_predictions)
    #start biding...
    for i in Train_df.index:
        current_bidprice = bidprices[i]
        #consider budget
        if current_budget <= current_bidprice:              
            #count lack budget times
            count_break = count_break + 1
#             print('lack of budget!',"current_budget:",current_budget,";current_bidprice:",current_bidprice,';count_break:',count_break)
            if count_break > 5:
                break  
            #reset bidprice
            current_bidprice = current_budget
        else:
            count_break = 0
            

        #Winning criterion:
        current_payprice = payprice[i]
        if current_bidprice > current_payprice:
            Train_df.at[i, 'win'] = int(1)
            current_budget = current_budget - current_payprice
        else:
            Train_df.at[i, 'win'] = int(0)

    #biding end        
    
    Train_df_new = Train_df[Train_df['win']!=0]

        
    return Train_df,current_budget

In [28]:
def calculate_click(base_bid,Train_df,payprice,test_predictions):
#     print('base_bid:',base_bid)
    result_df,current_budget = Prob3(Train_df,payprice,base_bid,test_predictions)
#     print('current_budget:',current_budget)
    Train_df_new = result_df[result_df['win']!=0]
    wintimes,_ = Train_df_new.shape
#     print('wintimes:',wintimes)
#     print('----------------')
    Clk = Train_df_new['click'].sum()    
    return Clk

In [75]:
#test range
base_bids = np.arange(80,260,10)#区间可以适当左右调整一下，但大概就是这个范围
count = 0
maxClks = []

In [35]:
for valid_result in valid_results[0:7]:
    count = count + 1
    print(str(count)+" valid result is being test...")
    test_predictions=valid_result['click_pred_per']
    Clks = [calculate_click(base_bid,Train_df_qr,payprice,test_predictions) for base_bid in base_bids]
    print("max click@",Clks.index(max(Clks)),"base_bid=",base_bids[Clks.index(max(Clks))],"click =",max(Clks))
    maxClks.append(max(Clks))
#     fig = plt.figure()
#     plt.plot(base_bids,Clks)
#     mpld3.display(fig)

1 valid result is being test...
max click@ 0 base_bid= 120 click = 129
2 valid result is being test...


KeyboardInterrupt: 

In [20]:
maxClks

[169, 168, 165, 167, 163, 168, 167, 167, 166, 169]

In [21]:
max(maxClks)

169

In [ ]:
#end

In [76]:
for i in range(50,60):
    count = count + 1
    print(str(count)+" valid result is being test...")
    test_predictions=valid_results[i]['click_pred_per']
    Clks = [calculate_click(base_bid,Train_df_qr,payprice,test_predictions) for base_bid in base_bids]
    print(Clks)
    print("================================================")
    print("max click@",Clks.index(max(Clks)),"base_bid=",base_bids[Clks.index(max(Clks))],"click =",max(Clks))
    maxClks.append(max(Clks))

1 valid result is being test...
[125, 125, 126, 130, 131, 132, 135, 137, 138, 138, 139, 139, 140, 140, 140, 140, 141, 141]
max click@ 16 base_bid= 240 click = 141
2 valid result is being test...
[118, 120, 124, 127, 129, 129, 130, 131, 133, 133, 134, 137, 138, 140, 141, 142, 143, 143]
max click@ 16 base_bid= 240 click = 143
3 valid result is being test...
[123, 127, 128, 128, 128, 128, 130, 134, 137, 139, 139, 139, 139, 139, 139, 141, 141, 141]
max click@ 15 base_bid= 230 click = 141
4 valid result is being test...
[119, 121, 128, 130, 130, 131, 133, 133, 133, 135, 137, 137, 139, 139, 140, 142, 142, 142]
max click@ 15 base_bid= 230 click = 142
5 valid result is being test...
[120, 124, 124, 126, 129, 129, 130, 130, 131, 135, 136, 136, 138, 139, 140, 141, 143, 144]
max click@ 17 base_bid= 250 click = 144
6 valid result is being test...
[122, 124, 125, 128, 129, 130, 131, 132, 132, 136, 137, 141, 143, 143, 143, 143, 144, 144]
max click@ 16 base_bid= 240 click = 144
7 valid result is bein

In [ ]:
1 valid result is being test...
[159, 158, 155, 153, 152, 149, 147, 147, 146, 144, 143, 142, 142, 143, 142, 142, 142, 140, 140, 140]
================================================
max click@ 0 base_bid= 160 click = 159
2 valid result is being test...
[154, 154, 154, 154, 155, 155, 156, 157, 157, 157, 157, 157, 157, 156, 155, 154, 153, 151, 151, 149]
================================================
max click@ 7 base_bid= 174 click = 157
3 valid result is being test...
[156, 157, 157, 158, 158, 158, 158, 158, 158, 158, 158, 157, 157, 156, 154, 154, 153, 152, 150, 147]
================================================
max click@ 3 base_bid= 166 click = 158
4 valid result is being test...
[155, 155, 155, 155, 156, 156, 156, 157, 157, 157, 157, 157, 157, 156, 156, 155, 153, 153, 152, 151]
================================================
max click@ 7 base_bid= 174 click = 157
5 valid result is being test...
[153, 153, 153, 154, 154, 154, 155, 155, 155, 156, 157, 158, 157, 156, 155, 154, 153, 152, 150, 148]
================================================
max click@ 11 base_bid= 182 click = 158

In [64]:
for i in range(15,20):
    count = count + 1
    print(str(count)+" valid result is being test...")
    test_predictions=valid_results[i]['click_pred_per']
    Clks = [calculate_click(base_bid,Train_df_qr,payprice,test_predictions) for base_bid in base_bids]
    print(Clks)
    print("================================================")
    print("max click@",Clks.index(max(Clks)),"base_bid=",base_bids[Clks.index(max(Clks))],"click =",max(Clks))
    maxClks.append(max(Clks))

1 valid result is being test...
[138, 139, 140, 140, 140, 142, 143, 143, 144, 147, 147, 151, 151, 152, 152, 152, 152, 153, 154, 154, 154, 156, 157]
max click@ 22 base_bid= 118 click = 157
2 valid result is being test...
[136, 136, 136, 137, 138, 139, 140, 141, 142, 143, 145, 145, 146, 146, 146, 147, 147, 148, 148, 148, 148, 149, 150]
max click@ 22 base_bid= 118 click = 150
3 valid result is being test...
[136, 136, 138, 140, 140, 140, 140, 140, 141, 141, 143, 143, 144, 145, 145, 145, 145, 145, 146, 146, 146, 146, 146]
max click@ 18 base_bid= 110 click = 146
4 valid result is being test...
[140, 140, 140, 140, 140, 141, 141, 141, 141, 142, 142, 142, 142, 143, 143, 143, 143, 143, 144, 145, 145, 145, 146]
max click@ 22 base_bid= 118 click = 146
5 valid result is being test...
[134, 135, 135, 135, 135, 136, 136, 136, 136, 137, 138, 138, 138, 139, 140, 141, 141, 141, 142, 142, 144, 144, 144]
max click@ 20 base_bid= 114 click = 144


In [ ]:
1 valid result is being test...
[157, 155, 152, 150, 150, 145, 144, 143, 142, 140, 140, 140, 138, 138, 136, 136, 135, 134, 133, 134, 133, 132, 131, 130, 129, 129, 128, 126, 126, 125, 125, 124, 123, 122, 122]
================================================
max click@ 0 base_bid= 95 click = 157
2 valid result is being test...
[153, 154, 155, 155, 155, 155, 155, 156, 157, 157, 157, 157, 158, 158, 158, 157, 155, 152, 152, 151, 146, 146, 147, 146, 146, 144, 144, 144, 143, 142, 144, 143, 142, 143, 142]
================================================
max click@ 12 base_bid= 107 click = 158
3 valid result is being test...
[155, 157, 157, 158, 159, 159, 159, 159, 160, 160, 160, 160, 161, 161, 161, 161, 162, 162, 162, 162, 162, 160, 160, 158, 157, 157, 153, 150, 151, 151, 149, 149, 148, 147, 149]
================================================
max click@ 16 base_bid= 111 click = 162
4 valid result is being test...
[158, 158, 158, 158, 158, 159, 159, 159, 159, 159, 159, 160, 160, 160, 160, 161, 161, 161, 161, 160, 159, 157, 156, 156, 154, 151, 150, 150, 150, 150, 149, 148, 148, 148, 148]
================================================
max click@ 15 base_bid= 110 click = 161
5 valid result is being test...
[153, 153, 154, 155, 156, 156, 156, 156, 156, 156, 156, 157, 157, 157, 158, 159, 159, 159, 159, 159, 159, 159, 160, 159, 159, 159, 156, 155, 156, 155, 150, 150, 150, 149, 147]
================================================
max click@ 22 base_bid= 117 click = 160
6 valid result is being test...
[154, 154, 154, 154, 155, 155, 155, 155, 156, 156, 157, 157, 157, 157, 157, 157, 157, 157, 158, 158, 159, 159, 159, 159, 159, 159, 159, 159, 160, 161, 161, 161, 161, 161, 160]
================================================
max click@ 29 base_bid= 124 click = 161
7 valid result is being test...
[156, 156, 156, 158, 159, 159, 159, 160, 160, 160, 160, 160, 160, 160, 161, 161, 161, 161, 161, 161, 161, 161, 161, 161, 162, 163, 162, 162, 161, 158, 158, 157, 156, 154, 150]
================================================
max click@ 25 base_bid= 120 click = 163
8 valid result is being test...
[155, 155, 156, 156, 156, 156, 156, 158, 158, 158, 158, 158, 158, 159, 161, 161, 162, 162, 162, 163, 163, 163, 164, 165, 165, 165, 165, 165, 166, 166, 166, 169, 169, 169, 168]
================================================
max click@ 31 base_bid= 126 click = 169
9 valid result is being test...
[154, 154, 155, 155, 157, 157, 158, 158, 159, 160, 160, 160, 160, 161, 161, 161, 161, 162, 162, 162, 162, 163, 163, 163, 164, 164, 164, 164, 164, 163, 161, 162, 159, 158, 159]
================================================
max click@ 24 base_bid= 119 click = 164
10 valid result is being test...
[156, 156, 156, 156, 156, 157, 157, 157, 157, 158, 158, 158, 158, 158, 158, 158, 158, 158, 158, 159, 159, 159, 160, 160, 160, 160, 160, 161, 161, 161, 160, 160, 159, 157, 158]
================================================
max click@ 27 base_bid= 122 click = 161

In [43]:
Clks

[147, 148, 149, 151, 154, 155, 156, 156, 155, 150]

In [ ]:
1 valid result is being test...
[149, 151, 151, 154, 155, 155, 155, 156, 156, 156, 157, 155, 152, 150, 150]
================================================
max click@ 10 base_bid= 95 click = 157
2 valid result is being test...
[145, 145, 147, 147, 149, 150, 150, 152, 152, 152, 153, 154, 155, 155, 155]
================================================
max click@ 12 base_bid= 97 click = 155
3 valid result is being test...
[147, 149, 149, 150, 151, 152, 153, 153, 154, 155, 155, 157, 157, 158, 159]
================================================
max click@ 14 base_bid= 99 click = 159

 随便看一下

In [13]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix


y_test = valid_result['click']
y_pred_class = valid_result['click_pred_class']

print(confusion_matrix(y_test, y_pred_class))

confusion = confusion_matrix(y_test, y_pred_class)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]
print ("TP:", TP)
print ("TN:", TN)
print ("FP:", FP)
print ("FN:", FN)

#准确率、识别率（Classification Accuracy）：分类器正确分类的比例
#print ((TP+TN) / float(TP+TN+FN+FP))
print ("Accuracy: %.6f%%" % (accuracy_score(y_test, y_pred_class)* 100.0) )

#灵敏性（Sensitivity），也称为真正例识别率、召回率（Recall）：正确识别的正例数据在实际正例数据中的百分比
#print (TP / float(TP+FN))
#recall = metrics.recall_score(y_test, y_pred_class)
print ("Recall: %.6f%%" % (recall_score(y_test, y_pred_class)* 100.0))

#特效性（Specificity），也称为真负例率：正确识别的负例数据在实际负例数据中的百分比
print ("Specificity: %.6f%%" % (TN / float(TN+FP)* 100.0))

#假阳率（False Positive Rate）：实际值是负例数据，预测错误的百分比
#print (FP / float(TN+FP))
specificity = TN / float(TN+FP)
print ("False Positive Rate: %.6f%%" %((1 - specificity)*100.0))

#精度（Precision）：看做精确性的度量，即标记为正类的数据实际为正例的百分比
#print (TP / float(TP+FP))
precision = precision_score(y_test, y_pred_class)
print ("Precision: %.6f%%" %(precision*100.0))

#F1（平衡F分数）：精确率和召回率的调和平均数
#print (TP / float(TP+FP))
f1 = f1_score(y_test, y_pred_class)
print ("F1 score: %.6f%%" %(f1*100.0))



[[303363    360]
 [   129     73]]
TP: 73
TN: 303363
FP: 360
FN: 129
Accuracy: 99.839105%
Recall: 36.138614%
Specificity: 99.881471%
False Positive Rate: 0.118529%
Precision: 16.859122%
F1 score: 22.992126%


## Test + Submission

In [ ]:
Test_df = pd.read_csv('/Users/weisihan/Downloads/we_data/mytestset.csv')
Group=  pd.read_csv('/Users/weisihan/Downloads/we_data/Group_xx.csv')

In [ ]:
Test_df = Test_df[['slotwidth', 'slotheight', 'slotprice', 'android', 'ios',
       'linux', 'mac', 'other', 'windows', 'chrome', 'firefox', 'ie',
       'maxthon', 'opera', 'other.1', 'safari', 'sogou', 'theworld',
       '10006', '10024', '10031', '10048', '10052', '10057', '10059',
       '10063', '10067', '10074', '10075', '10076', '10077', '10079',
       '10083', '10093', '10102', '10110', '10111', '10114', '10115',
       '10116', '10117', '10118', '10120', '10123', '10125', '10126',
       '10127', '10129', '10130', '10131', '10133', '10138', '10140',
       '10142', '10145', '10146', '10147', '10148', '10149', '10684',
       '11092', '11278', '11379', '11423', '11512', '11576', '11632',
       '11680', '11724', '11944', '13042', '13403', '13496', '13678',
       '13776', '13800', '13866', '13874', '14273', '15398', '16593',
       '16617', '16661', '16706', '16751', '16753', 'hour_sin', 'hour_cos',
       'city_per', 'region_per', 'weekday_0', 'weekday_1', 'weekday_2',
       'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6',
       'adexchange_1.0', 'adexchange_2.0', 'adexchange_3.0',
       'adexchange_4.0', 'adexchange_nan', 'slotvisibility_0',
       'slotvisibility_1', 'slotvisibility_2', 'slotvisibility_255',
       'slotvisibility_FifthView', 'slotvisibility_FirstView',
       'slotvisibility_FourthView', 'slotvisibility_Na',
       'slotvisibility_OtherView', 'slotvisibility_SecondView',
       'slotvisibility_ThirdView', 'slotformat_0', 'slotformat_1',
       'slotformat_5', 'slotformat_Na', 'advertiser_1458',
       'advertiser_2259', 'advertiser_2261', 'advertiser_2821',
       'advertiser_2997', 'advertiser_3358', 'advertiser_3386',
       'advertiser_3427', 'advertiser_3476']]

In [ ]:
test_x = Test_df
test_predictions = model.predict_proba(test_x)
test_predictions_class = model.predict(test_x)

In [ ]:
test_result = pd.DataFrame(columns=['bidprice', 'click_pred_per', 'click_pred_class'])  
test_result['click_pred_per'] = test_predictions[:,1]
test_result['click_pred_class'] = test_predictions_class
test_result.head()

In [ ]:
test_result['click_pred_per']=recalibrate(test_result['click_pred_per'],Samp_per)
test_result.head()

In [ ]:
base_bid=133
#这个变成你测出来最高clk的basebid！！！！！！！
avgCTR = 1793/2430981#train set
test_result['bidprice'] = base_bid*test_result['click_pred_per']/avgCTR
test_result.head()

In [ ]:
Group['bidprice']=test_result['bidprice']
Group.head()

In [ ]:
Group.to_csv('/Users/weisihan/Desktop/Group_6.csv', encoding='utf-8', index=False)